In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity #This we use to recommend !

In [ ]:
data = pd.read_csv("netflixData.csv")
print(data.head())

                                Show Id                          Title  \
0  cc1b6ed9-cf9e-4057-8303-34577fb54477                       (Un)Well   
1  e2ef4e91-fb25-42ab-b485-be8e3b23dedb                         #Alive   
2  b01b73b7-81f6-47a7-86d8-acb63080d525  #AnneFrank - Parallel Stories   
3  b6611af0-f53c-4a08-9ffa-9716dc57eb9c                       #blackAF   
4  7f2d4170-bab8-4d75-adc2-197f7124c070               #cats_the_mewvie   

                                         Description  \
0  This docuseries takes a deep dive into the luc...   
1  As a grisly virus rampages a city, a lone man ...   
2  Through her diary, Anne Frank's story is retol...   
3  Kenya Barris and his family navigate relations...   
4  This pawesome documentary explores how our fel...   

                      Director  \
0                          NaN   
1                       Cho Il   
2  Sabina Fedeli, Anna Migotto   
3                          NaN   
4             Michael Margolis   

             

In [ ]:
print(data.isnull().sum())

Show Id                  0
Title                    0
Description              0
Director              2064
Genres                   0
Cast                   530
Production Country     559
Release Date             3
Rating                   4
Duration                 3
Imdb Score             608
Content Type             0
Date Added            1335
dtype: int64


In [ ]:
data = data[["Title", "Description", "Content Type", "Genres"]]
print(data.head())

                           Title  \
0                       (Un)Well   
1                         #Alive   
2  #AnneFrank - Parallel Stories   
3                       #blackAF   
4               #cats_the_mewvie   

                                         Description Content Type  \
0  This docuseries takes a deep dive into the luc...      TV Show   
1  As a grisly virus rampages a city, a lone man ...        Movie   
2  Through her diary, Anne Frank's story is retol...        Movie   
3  Kenya Barris and his family navigate relations...      TV Show   
4  This pawesome documentary explores how our fel...        Movie   

                                           Genres  
0                                      Reality TV  
1  Horror Movies, International Movies, Thrillers  
2             Documentaries, International Movies  
3                                     TV Comedies  
4             Documentaries, International Movies  


In [ ]:
print(data.isnull().sum())

Title           0
Description     0
Content Type    0
Genres          0
dtype: int64


In [ ]:
data = data.dropna()

In [ ]:
data

,Title,Description,Content Type,Genres
0,(Un)Well,This docuseries takes a deep dive into the luc...,TV Show,Reality TV
1,#Alive,"As a grisly virus rampages a city, a lone man ...",Movie,"Horror Movies, International Movies, Thrillers"
2,#AnneFrank - Parallel Stories,"Through her diary, Anne Frank's story is retol...",Movie,"Documentaries, International Movies"
3,#blackAF,Kenya Barris and his family navigate relations...,TV Show,TV Comedies
4,#cats_the_mewvie,This pawesome documentary explores how our fel...,Movie,"Documentaries, International Movies"
...,...,...,...,...
5962,الف مبروك,"On his wedding day, an arrogant, greedy accoun...",Movie,"Comedies, Dramas, International Movies"
5963,دفعة القاهرة,A group of women leaves Kuwait to attend unive...,TV Show,"International TV Shows, TV Dramas"
5964,海的儿子,"Two brothers start a new life in Singapore, wh...",TV Show,"International TV Shows, TV Dramas"
5965,반드시 잡는다,After people in his town start turning up dead...,Movie,"Dramas, International Movies, Thrillers"


In [ ]:
import nltk    #NLP Library
import re    #Regular expression
nltk.download('stopwords')   #Download the extension
stemmer = nltk.SnowballStemmer("english")  #create stemmer method
from nltk.corpus import stopwords   #load stopwords
import string
stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)

    text = [word for word in text.split(' ') if word not in stopwords]
    text=" ".join(text)

#data["Title"] = data["Title"].apply(clean)


In [ ]:
print(data.Title.sample(10))
#print(data.Title)

1911       Guatemala: Heart of the Mayan World
3843                              Rainbow Time
2454                                     K-On!
2044                             Hidden Worlds
2465                    Kabhi Alvida Naa Kehna
2767                                    Locust
391     Anthony Jeselnik: Thoughts and Prayers
949              Cheech & Chong's Still Smokin
1766                                 Gangaajal
704                                 Black Spot
Name: Title, dtype: object


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
feature = data['Genres'].tolist()

#Create an instance of TF-IDF-Vectorizer
tfidf = TfidfVectorizer(stop_words="english")

# Fit and transform the vectorizer on our corpus
tfidf_matrix = tfidf.fit_transform(feature)

#Compute the cosine similarity matrix
similarity = cosine_similarity(tfidf_matrix)

indices = pd.Series(data.index,
                    index=data['Title']).drop_duplicates()

In [ ]:


def netFlix_recommendation(title, num_recommendations=10):
    if title not in indices:
        return f"{title} is not in the database"

    index = indices[title]  # Find index of given movie
    similarity_scores = list(enumerate(similarity[index]))  # Get similarity scores
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)  # Sort by similarity
    similarity_scores = similarity_scores[1:num_recommendations+1]  # Get top N recommendations (skip itself)

    movie_indices = [i[0] for i in similarity_scores]  # Extract movie indices
    return data['Title'].iloc[movie_indices].tolist()  # Return list of recommended movies




In [ ]:
# Example Usage:
print(netFlix_recommendation("#Alive", num_recommendations=5))


['Aaviri', 'Andhaghaaram', 'Andhakaaram', 'Apostle', 'Game Over (Hindi Version)']
